In [1]:
# docker run -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" docker.elastic.co/elasticsearch/elasticsearch:7.7.0

In [1]:
import numpy as np

## Crawl 

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

from paper_crawler.paper_crawler.spiders.semanticscholar import (
    SemanticscholarSpider,
)

In [3]:
process = CrawlerProcess(
    settings={
        "FEEDS": {"papers.json": {"format": "json"}},
        "LOG_ENABLED": False
        #     "LOG_LEVEL": 'INFO',
    }
)
process.crawl(SemanticscholarSpider, max_papers=2000)
process.start()

## Load Crawled Data

In [43]:
import json

with open("papers.json") as f:
    items = json.load(f)

In [4]:
len(items)

2006

## Elasticsearch 

In [5]:
from elasticsearch import Elasticsearch

### Connect 

In [36]:
es = Elasticsearch(hosts=[{"host": "localhost", "port": 9200}])

### Clear previous index 

In [37]:
es.indices.delete("paper-index", ignore=404)

{'error': {'root_cause': [{'type': 'index_not_found_exception',
    'reason': 'no such index [paper-index]',
    'resource.type': 'index_or_alias',
    'resource.id': 'paper-index',
    'index_uuid': '_na_',
    'index': 'paper-index'}],
  'type': 'index_not_found_exception',
  'reason': 'no such index [paper-index]',
  'resource.type': 'index_or_alias',
  'resource.id': 'paper-index',
  'index_uuid': '_na_',
  'index': 'paper-index'},
 'status': 404}

### Create new index 

In [38]:
es.indices.create(index="paper-index", ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'paper-index'}

### Create persistent layer 

In [39]:
from elasticsearch_dsl import (
    Document,
    Date,
    Nested,
    Boolean,
    analyzer,
    InnerDoc,
    Completion,
    Keyword,
    Text,
    Integer,
    Float,
)

In [40]:
class Paper(Document):
    title = Text(fields={"raw": Keyword()})
    date = Integer()
    abstract = Text()
    authors = Text()
    references = Text()
    page_rank = Float()

    class Index:
        name = "paper-index"

In [41]:
# create the mappings in Elasticsearch
Paper.init(using=es)

### Insert items 

In [16]:
from elasticsearch import TransportError
from elasticsearch.helpers import bulk

#### Solution #1

In [44]:
# https://elasticsearch-py.readthedocs.io/en/master/helpers.html#bulk-helpers
# https://www.elastic.co/guide/en/elasticsearch/reference/master/docs-bulk.html


def gendata():
    for idx, item in enumerate(items):
        if item["date"] == "" or not item["date"].isdigit():
            del item["date"]
        yield {
            "_index": "paper-index",
            "_id": item["id"],
            "page_rank": 1.0,
            **item,
        }


bulk(es, gendata())

(2006, [])

#### Solution #2

In [ ]:
for item in items:
    paper = Paper(meta={"id": item["id"]}, page_rank=1.0, **item)
    paper.save(using=es)

### Funcions for inserting items and clearing index

In [231]:
def gen_utils(host=None):
    if host is None:
        host = {"host": "localhost", "port": 9200}
    es = Elasticsearch(hosts=[{"host": "localhost", "port": 9200}])

    def clear_index():
        es.indices.delete("paper-index", ignore=404)
        es.indices.create(index="paper-index", ignore=400)

    def insert_items(items):
        for item in items:
            paper = Paper(meta={"id": item["id"]}, **item)
            paper.save(using=es)

    return clear_index, insert_items

In [66]:
clear, insert = gen_utils()

In [67]:
clear()

In [68]:
insert(items)

## Calculating page rank

In [18]:
import numpy as np

In [19]:
all_papers = list(items)

In [20]:
all_ids = sorted(list(map(lambda x: x["id"], all_papers)))

In [21]:
p_matrix = np.zeros((len(all_ids), len(all_ids)))

In [22]:
id_loc = dict()
for index, paper_id in enumerate(all_ids):
    id_loc[paper_id] = index

In [23]:
for index, paper_id in enumerate(all_ids):
    paper = Paper.get(id=paper_id, using=es)
    if paper.references is not None:
        for reference_id in paper.references:
            try:
                p_matrix[index, id_loc[reference_id]] = 1
            except KeyError:
                continue

In [24]:
alpha = 0.1
N = len(all_ids)
v = np.ones((1, N))

In [25]:
row_sums = np.sum(p_matrix, axis=1, keepdims=True)

In [26]:
# first part is for rows having nonzero elements
# second part is for dead-ends
p_matrix = ((row_sums > 0) * 1) * (
    (1 - alpha) * p_matrix / (row_sums + np.logical_not(row_sums > 0) * 1)
    + alpha * v / N
) + (np.logical_not(row_sums > 0) * 1) * v / N

In [27]:
x0 = np.ones((1, N)) / N

In [28]:
while True:
    next_state = x0 @ p_matrix
    if np.allclose(next_state, x0, rtol=0.0001):
        break
    x0 = next_state

In [29]:
next_state = next_state * 1000

In [ ]:
for index, paper_id in enumerate(all_ids):
    paper = Paper.get(id=paper_id, using=es)
    paper.update(page_rank=next_state[0, index], using=es)
    paper.save(using=es)

#### Solution #2 

In [45]:
def gendata():
    for index, paper_id in enumerate(all_ids):
        yield {
            "_index": "paper-index",
            "_id": items[index]["id"],
            "_source": {"doc": {"page_rank": next_state[0, index]}},
            "_op_type": "update",
        }


bulk(es, gendata())

(2006, [])

#### Solution #3

In [307]:
# 429
body = ""
for index, paper_id in enumerate(all_ids):
    body += f"""{{ "update" : {{"_id" : "{paper_id}", "_index" : "paper-index"}} }}
{{ "doc" : {{"page_rank" : {next_state[0, index]} }}}}
"""
    break
body += ""
res = es.bulk(body)
res

## Search 

In [46]:
from elasticsearch_dsl import Search

In [66]:
title_search = ""
# title_search = "Recurrent Models with Fast-Forward"
# abstract_search = 'different "thinned" networks'
abstract_search = ""
year_search = 0
title_weight = 1
abstract_weight = 100
year_weight = 1

In [67]:
def search(
    title_search: str,
    abstract_search: str,
    year_search: int,
    title_weight: float,
    abstract_weight: float,
    year_weight: float,
    apply_page_rank: bool = True,
):
    if apply_page_rank:
        return es.search(
            index="paper-index",
            body={
                "query": {
                    # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-function-score-query.html#function-weight
                    "function_score": {
                        "query": {
                            "bool": {
                                "should": [
                                    {"match": {"title": title_search}},
                                    {"match": {"abstract": abstract_search}},
                                    {"range": {"date": {"gte": year_search}}},
                                ]
                            }
                        },
                        "functions": [
                            {
                                "filter": {"match": {"title": title_search}},
                                "weight": title_weight,
                            },
                            {
                                "filter": {
                                    "match": {"abstract": abstract_search}
                                },
                                "weight": abstract_weight,
                            },
                            # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html
                            {
                                "filter": {
                                    "range": {"date": {"gte": year_search}}
                                },
                                "weight": year_weight,
                            },
                            # https://www.elastic.co/guide/en/elasticsearch/reference/current/static-scoring-signals.html
                            # https://www.elastic.co/guide/en/elasticsearch/reference/7.x/query-dsl-rank-feature-query.html#rank-feature-query-saturation
                            {
                                "script_score": {
                                    "script": {
                                        #                                 "source": "_score * saturation(doc['page_rank'].value, 10)"
                                        "source": "_score * doc['page_rank'].value"
                                    }
                                }
                            },
                        ],
                    }
                }
            },
        )
    return es.search(
        index="paper-index",
        body={
            "query": {
                # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-function-score-query.html#function-weight
                "function_score": {
                    "query": {
                        "bool": {
                            "should": [
                                {"match": {"title": title_search}},
                                {"match": {"abstract": abstract_search}},
                                {"range": {"date": {"gte": year_search}}},
                            ]
                        }
                    },
                    "functions": [
                        {
                            "filter": {"match": {"title": title_search}},
                            "weight": title_weight,
                        },
                        {
                            "filter": {"match": {"abstract": abstract_search}},
                            "weight": abstract_weight,
                        },
                        # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html
                        {
                            "filter": {
                                "range": {"date": {"gte": year_search}}
                            },
                            "weight": year_weight,
                        },
                    ],
                }
            }
        },
    )

In [68]:
res = search(
    title_search,
    abstract_search,
    year_search,
    title_weight,
    abstract_weight,
    year_weight,
    apply_page_rank=True,
)
for hit in res["hits"]["hits"]:
    print(
        f'{hit["_source"]["title"]}\n\t Score: {hit["_score"]}\n\t Year: {hit["_source"]["date"]}'
    )

Improving Neural Networks with Dropout
	 Score: 9083.909
	 Year: 2013
Dropout: a simple way to prevent neural networks from overfitting
	 Score: 6437.26
	 Year: 2014
Learning Networks of Neurons with Boolean Logic
	 Score: 6280.1772
	 Year: 1987
Neural Network Classifiers Estimate Bayesian a posteriori Probabilities
	 Score: 4665.441
	 Year: 1991
Improving the speed of neural networks on CPUs
	 Score: 3927.0295
	 Year: 2011
Exploring Strategies for Training Deep Neural Networks
	 Score: 3711.072
	 Year: 2009
The Aim of Inductive Logic
	 Score: 3299.706
	 Year: 1966
Speech recognition with deep recurrent neural networks
	 Score: 2925.1804
	 Year: 2013
Spatial Pyramid Pooling in Deep Convolutional Networks for Visual Recognition
	 Score: 2786.2263
	 Year: 2015
BPS: a learning algorithm for capturing the dynamic nature of speech
	 Score: 2398.877
	 Year: 1989


In [69]:
res = search(
    title_search,
    abstract_search,
    year_search,
    title_weight,
    abstract_weight,
    year_weight,
    apply_page_rank=False,
)
for hit in res["hits"]["hits"]:
    print(
        f'{hit["_source"]["title"]}\n\t Score: {hit["_score"]}\n\t Year: {hit["_source"]["date"]}'
    )

Improving Neural Networks with Dropout
	 Score: 1603.5023
	 Year: 2013
Dropout: a simple way to prevent neural networks from overfitting
	 Score: 1566.7781
	 Year: 2014
Task Decomposition Through Competition in a Modular Connectionist Architecture: The What and Where Vision Tasks
	 Score: 806.23944
	 Year: 1991
Actively Searching for an Effective Neural Network Ensemble
	 Score: 649.9195
	 Year: 1996
Sluice networks: Learning what to share between loosely related tasks
	 Score: 634.2689
	 Year: 2017
Structured Attention Networks
	 Score: 634.2689
	 Year: 2017
Best practices for convolutional neural networks applied to visual document analysis
	 Score: 632.9023
	 Year: 2003
Convolutional neural networks applied to house numbers digit classification
	 Score: 632.8181
	 Year: 2012
Training and Analysing Deep Recurrent Neural Networks
	 Score: 626.65784
	 Year: 2013
Distributed Deep Learning Using Synchronous Stochastic Gradient Descent
	 Score: 598.5747
	 Year: 2016


In [53]:
# With page rank
res = es.search(
    index="paper-index",
    body={
        "query": {
            # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-function-score-query.html#function-weight
            "function_score": {
                "functions": [
                    {
                        "filter": {"match": {"title": title_search}},
                        "weight": title_weight,
                    },
                    {
                        "filter": {"match": {"abstract": abstract_search}},
                        "weight": abstract_weight,
                    },
                    # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html
                    {
                        "filter": {"range": {"date": {"gte": year_search}}},
                        "weight": year_weight,
                    },
                    # https://www.elastic.co/guide/en/elasticsearch/reference/current/static-scoring-signals.html
                    # https://www.elastic.co/guide/en/elasticsearch/reference/7.x/query-dsl-rank-feature-query.html#rank-feature-query-saturation
                    {
                        "script_score": {
                            "script": {
                                #                                 "source": "_score * saturation(doc['page_rank'].value, 10)"
                                "source": "_score * doc['page_rank'].value"
                            }
                        }
                    },
                ]
            }
        }
    },
)

In [54]:
for hit in res["hits"]["hits"]:
    print(f'{hit["_source"]["title"]}: {hit["_score"]}')

The Lottery Ticket Hypothesis: Training Pruned Neural Networks: 0.0
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding: 0.0
Optimal Brain Surgeon and general network pruning: 0.0
Understanding Dropout: 0.0
Diversity Networks: Neural Network Compression Using Determinantal Point Processes: 0.0
Data-free Parameter Pruning for Deep Neural Networks: 0.0
A Deep Neural Network Compression Pipeline: Pruning, Quantization, Huffman Encoding: 0.0
Pruning Filters for Efficient ConvNets: 0.0
Deep Compression: Compressing Deep Neural Network with Pruning, Trained Quantization and Huffman Coding: 0.0
ThiNet: A Filter Level Pruning Method for Deep Neural Network Compression: 0.0


In [38]:
# Without page rank
res = es.search(
    index="paper-index",
    body={
        "query": {
            # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-function-score-query.html#function-weight
            "function_score": {
                "functions": [
                    {
                        "filter": {"match": {"title": title_search}},
                        "weight": title_weight,
                    },
                    {
                        "filter": {"match": {"abstract": abstract_search}},
                        "weight": abstract_weight,
                    },
                    # https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-range-query.html
                    {
                        "filter": {"range": {"date": {"gte": year_search}}},
                        "weight": year_weight,
                    },
                ]
            }
        }
    },
)

In [39]:
for hit in res["hits"]["hits"]:
    print(f'{hit["_source"]["title"]}: {hit["_score"]}')

The Lottery Ticket Hypothesis: Training Pruned Neural Networks: 100.0
BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding: 50.0
Character-Level Language Modeling with Deeper Self-Attention: 50.0
U-Net: Machine Reading Comprehension with Unanswerable Questions: 50.0
GLUE: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding: 50.0
Memory Architectures in Recurrent Neural Network Language Models: 50.0
Annotation Artifacts in Natural Language Inference Data: 50.0
Transforming Question Answering Datasets Into Natural Language Inference Datasets: 50.0
Visual Referring Expression Recognition: What Do Systems Actually Learn?: 50.0
Visual Dialog: 50.0


#### Experiments 

In [272]:
s = Search(using=es, index="paper-index").query(
    "match", abstrat="different thinned networks"
)

In [189]:
s = Search(using=es, index="paper-index").query("match", id="323694313")

In [273]:
response = s.execute()

In [274]:
for hit in response:
    print(hit.meta.score, hit.title)

In [75]:
# by calling .search we get back a standard Search object
s = Paper.search(using=es)
# the search is already limited to the index and doc_type of our document
s = s.query("match", title="the lottery")

In [76]:
response = s.execute()

In [77]:
for hit in response:
    print(hit.meta.score, hit.title, hit.page_rank)

9.071335 The Lottery Ticket Hypothesis: Training Pruned Neural Networks 0.0002729904488451375
2.595128 The structure of the “THE”-multiprogramming system 0.0003314600978953064
2.3790498 The Perception of the Visual World 0.00040618841508893987
2.3790498 Computing the Maximum and the Median 0.0003331176564811347
2.2939515 The Stanford Cart and the CMU Rover 0.0002858594703635938
2.2866845 The Randomization Bases of the Problem of the Amalgamation of Weighted Means 0.0002637217548452573
2.2830682 For Valid Generalization the Size of the Weights is More Important than the Size of the Network 0.0029864949725032395
2.24891 The Sample Complexity of Pattern Classification with Neural Networks: The Size of the Weights is More Important than the Size of the Network 0.0002745209411868165
2.2335901 Modeling the Shape of the Scene: A Holistic Representation of the Spatial Envelope 0.00035578959729973584
2.2147312 The CNN is universal as the Turing machine 0.00035804376339981716


### HITS 

In [63]:
from bidict import bidict

In [64]:
papers = items
# Give each author an id and store them in this dict
author_ids = bidict()
id_counter = 0
for paper in papers:
    paper_authors = paper["authors"]
    for author in paper_authors:
        if author not in author_ids:
            author_ids[author] = id_counter
            id_counter += 1

# Map each paper to it's authors' ids
paper_authors_dict = dict()
for paper in papers:
    paper_id = paper["id"]
    paper_authors = paper["authors"]
    paper_authors_dict[paper_id] = []
    for author in paper_authors:
        author_id = author_ids[author]
        paper_authors_dict[paper_id].append(author_id)

# Map each author to the authors he/she has referenced
author_references = dict()
for paper in papers:
    paper_authors = paper["authors"]
    paper_references = paper["references"]
    references = []
    for reference_id in paper_references:
        if reference_id in paper_authors_dict:
            references += paper_authors_dict[reference_id]
    for author in paper_authors:
        author_id = author_ids[author]
        if author_id not in author_references:
            author_references[author_id] = []
        author_references[author_id] += references

In [65]:
num_authors = len(author_ids)

In [66]:
connectivity_matrix = np.zeros((num_authors, num_authors))
for author, references in author_references.items():
    for reference in references:
        connectivity_matrix[author, reference] = 1

In [67]:
a = np.ones(num_authors)
h = np.ones(num_authors)

In [68]:
for _ in range(5):
    for i in range(num_authors):
        h[i] = np.sum(connectivity_matrix[i, :] * a)
    for i in range(num_authors):
        a[i] = np.sum(connectivity_matrix[:, i].T * h)
    a /= np.sum(a)
    h /= np.sum(h)

In [69]:
best_authors = []
for i in np.argpartition(a, -10)[-10:]:
    best_authors.append((author_ids.inverse[i], a[i]))

In [70]:
print("Best Authors:")
best_authors = sorted(best_authors, key=lambda x: -x[1])
for author, authority in best_authors:
    print(f"{author} : {authority}")

Best Authors:
Ilya Sutskever : 0.01332247665339998
Geoffrey E. Hinton : 0.012897100909998679
Quoc V. Le : 0.010406961290190133
Yoshua Bengio : 0.01031043642665406
Yann LeCun : 0.010073318403498548
Rob Fergus : 0.009878935870591975
Alex Krizhevsky : 0.009769722710715257
Andrew Y. Ng : 0.009080287367886264
Oriol Vinyals : 0.008076150300655045
Marc'Aurelio Ranzato : 0.007608169735770792


## Ranking SVM 

In [60]:
from sklearn.metrics import accuracy_score

### Read Data 

In [61]:
with open("./MIR_Phase3/data/train.txt", "r") as f:
    train_contents = f.read()
with open("./MIR_Phase3/data/vali.txt", "r") as f:
    val_contents = f.read()
with open("./MIR_Phase3/data/test.txt", "r") as f:
    test_contents = f.read()

### Parse Data 

In [62]:
from dataclasses import dataclass
import numpy as np


@dataclass
class QueryResult:
    qid: int
    doc_id: str
    relevance: int
    feature_vector: np.array

In [63]:
from typing import Dict, List


def parse_data(data: str) -> Dict[int, List[QueryResult]]:
    query_to_result_dict = dict()
    for line in data.split("\n"):
        splitted_line = line.split()
        try:
            relevance = int(splitted_line[0])
            qid = int(line.split()[1][4:])
            doc_id = line.split()[50]
            feature_vector = []
            for feature in line.split()[2:48]:
                feature_vector.append(float(feature.split(":")[1]))
            if qid not in query_to_result_dict:
                query_to_result_dict[qid] = []
            query_to_result_dict[qid].append(
                QueryResult(qid, doc_id, relevance, np.array(feature_vector))
            )
        except IndexError:
            pass
    return query_to_result_dict

In [64]:
train_query_to_result_dict = parse_data(train_contents)
val_query_to_result_dict = parse_data(val_contents)
test_query_to_result_dict = parse_data(test_contents)

### Prepare Data for Training 

In [65]:
POSITIVE_CLASS = 1
NEGATIVE_CLASS = 0

In [66]:
def prepare_data_for_model(query_to_result_dict: Dict[int, List[QueryResult]]):
    X = []
    Y = []
    for qid, results in query_to_result_dict.items():
        # Sort in decresing order
        results = sorted(results, key=lambda x: -x.relevance)
        num_results = len(results)
        for idx, result in enumerate(results):
            for other_result in results[idx + 1 : -1]:
                if result.relevance > other_result.relevance:
                    positive_vector = (
                        result.feature_vector - other_result.feature_vector
                    )
                    X.append(positive_vector)
                    X.append(-positive_vector)
                    Y.append(POSITIVE_CLASS)
                    Y.append(NEGATIVE_CLASS)
    return X, Y

In [67]:
X_train, y_train = prepare_data_for_model(train_query_to_result_dict)
X_val, y_val = prepare_data_for_model(val_query_to_result_dict)

### Check Hyperparameters 

In [68]:
# My laptop melts with all the data
num_train = len(X_train) // 10
X_train, y_train = X_train[:num_train], y_train[:num_train]

In [69]:
from sklearn.svm import SVC

In [25]:
settings = [
    {"kernel": "rbf", "C": 1.0},
    {"kernel": "linear", "C": 1.0},
    {"kernel": "linear", "C": 10.0},
    {"kernel": "linear", "C": 0.1},
]
for setting in settings:
    print(setting)
    clf = SVC(**setting)
    clf.fit(X_train, y_train)
    y_pred_val = clf.predict(X_val)
    print(accuracy_score(y_val, y_pred_val))

{'kernel': 'rbf', 'C': 1.0}
0.7668227033352838
{'kernel': 'linear', 'C': 1.0}
0.7753803393797543
{'kernel': 'linear', 'C': 10.0}
0.7746123464014043
{'kernel': 'linear', 'C': 0.1}
0.7777208894090111


### Train Model 

In [70]:
X_train, y_train = prepare_data_for_model(train_query_to_result_dict)
X_val, y_val = prepare_data_for_model(val_query_to_result_dict)
clf = SVC(kernel="linear")
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

### Test Model 

In [77]:
def compare(doc1, doc2):
    return (
        1
        if clf.predict([doc1.feature_vector - doc2.feature_vector])[0]
        == POSITIVE_CLASS
        else -1
    )

In [87]:
# Python program for implementation of Quicksort Sort

# This function takes last element as pivot, places
# the pivot element at its correct position in sorted
# array, and places all smaller (smaller than pivot)
# to left of pivot and all greater elements to right
# of pivot
def partition(arr, low, high):
    i = low - 1  # index of smaller element
    pivot = arr[high]  # pivot

    for j in range(low, high):

        # If current element is smaller than or
        # equal to pivot
        if compare(pivot, arr[j]) == -1:

            # increment index of smaller element
            i = i + 1
            arr[i], arr[j] = arr[j], arr[i]

    arr[i + 1], arr[high] = arr[high], arr[i + 1]
    return i + 1


# The main function that implements QuickSort
# arr[] --> Array to be sorted,
# low  --> Starting index,
# high  --> Ending index

# Function to do Quick sort
def quickSort(arr, low, high):
    if low < high:

        # pi is partitioning index, arr[p] is now
        # at right place
        pi = partition(arr, low, high)

        # Separately sort elements before
        # partition and after partition
        quickSort(arr, low, pi - 1)
        quickSort(arr, pi + 1, high)

In [88]:
from src.metrics import ndcg_at_k

In [89]:
evals = []
for qid, results in test_query_to_result_dict.items():
    gt = [x.relevance for x in sorted(results, key=lambda x: -x.relevance)]
    quickSort(results, 0, len(results) - 1)
    pred = [x.relevance for x in results]
    evals.append(ndcg_at_k(pred, gt, 5))

In [90]:
np.average(evals)

0.4403820991960565